# 15-10. 프로젝트: 단어 level로 번역기 업그레이드하기
* 데이터에서 상위 33000개의 샘플만 사용할 것
* 33000개 중 3000개는 테스트 데이터로 분리할 것

## 필요한 모듈 import

In [1]:
# import os
import re

import pandas as pd
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

## 1. 정제, 정규화, 전처리

In [2]:
# 파일 불러오기
file_path = 'data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수 :',len(lines))

# 세 번째 열 제거하고, 33000개만 사용
lines = lines[['eng', 'fra']][:33000]
lines.head(10)

전체 샘플의 수 : 178009


,eng,fra
0,Go.,Va !
1,Hi.,Salut !
2,Hi.,Salut.
3,Run!,Cours !
4,Run!,Courez !
5,Who?,Qui ?
6,Wow!,Ça alors !
7,Fire!,Au feu !
8,Help!,À l'aide !
9,Jump.,Saute.


In [3]:
# 전처리 1: 구두점을 단어와 분리 (. / , / ! / ?)
# 전처리 2: 소문자로 변환
for idx, val in enumerate(lines['eng']):
    lines['eng'][idx] = val.lower()
    # lines['eng'][idx] = lines['eng'][idx].replace('.', ' .').replace('?', ' ?').replace('!', ' !').replace(',', ' ,')
    lines['eng'][idx] = re.sub(r"([?.!,¿])", r" \1 ", lines['eng'][idx])
    lines['eng'][idx] = re.sub(r"[^a-zA-Z!.?]+", r" ", lines['eng'][idx])
    lines['eng'][idx] = re.sub(r"\s+", r" ", lines['eng'][idx])
    
for idx, val in enumerate(lines['fra']):
    lines['fra'][idx] = val.lower()
    # lines['fra'][idx] = lines['fra'][idx].replace('.', ' .').replace('?', ' ?').replace('!', ' !').replace(',', ' ,')
    lines['fra'][idx] = re.sub(r"([?.!,¿])", r" \1 ", lines['fra'][idx])
    lines['fra'][idx] = re.sub(r"[^a-zA-Z!.?]+", r" ", lines['fra'][idx])
    lines['fra'][idx] = re.sub(r"\s+", r" ", lines['fra'][idx])
    
lines.head(10)

,eng,fra
0,go .,va !
1,hi .,salut !
2,hi .,salut .
3,run !,cours !
4,run !,courez !
5,who ?,qui ?
6,wow !,a alors !
7,fire !,au feu !
8,help !,l aide !
9,jump .,saute .


## 2. 디코더의 문장에 시작 토큰과 종료 토큰 넣기

In [4]:
sos_token = '<sos>'
eos_token = '<eos>'
lines.fra = lines.fra.apply(lambda x : sos_token + ' '+ x + ' ' + eos_token)
print('전체 샘플의 수 :',len(lines))
lines.sample(5)

전체 샘플의 수 : 33000


,eng,fra
26739,tom drank lemonade .,<sos> tom a bu de la limonade . <eos>
20164,is that your wife ?,<sos> est ce votre femme ? <eos>
4143,it was clean .,<sos> ce fut net . <eos>
28845,hand in your papers .,<sos> remettez leur vos papiers . <eos>
22465,you did well tom .,<sos> vous avez bien fait tom . <eos>


## 3. 케라스의 토크나이저로 텍스트를 숫자로 바꾸기

In [5]:
# 단어 단위로 토크나이저 생성 후 숫자로 변환

# eng
eng_tokenizer = Tokenizer(filters="", lower=False) # num_words=7000
eng_tokenizer.fit_on_texts(lines.eng)
input_text = eng_tokenizer.texts_to_sequences(lines.eng)
input_text[:5]

[[30, 1], [1132, 1], [1132, 1], [260, 32], [260, 32]]

In [6]:
# fra
fra_tokenizer = Tokenizer(filters="", lower=False) # num_words=7000
fra_tokenizer.fit_on_texts(lines.fra)
target_text = fra_tokenizer.texts_to_sequences(lines.fra)
target_text[:5]

[[1, 91, 12, 2],
 [1, 1068, 12, 2],
 [1, 1068, 3, 2],
 [1, 928, 12, 2],
 [1, 2190, 12, 2]]

In [7]:
# 단어장 크기 저장
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1

print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)

영어 단어장의 크기 : 4662
프랑스어 단어장의 크기 : 7326


In [8]:
# 시작 토큰, 종료 토큰 제거
encoder_input = input_text

decoder_input = [[ char for char in line if char != fra_tokenizer.word_index[eos_token] ] for line in target_text]
decoder_target = [[ char for char in line if char != fra_tokenizer.word_index[sos_token] ] for line in target_text]

print(decoder_input[:3])
print(decoder_target[:3])

[[1, 91, 12], [1, 1068, 12], [1, 1068, 3]]
[[91, 12, 2], [1068, 12, 2], [1068, 3, 2]]


In [9]:
# 최대 길이
max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])

print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

영어 시퀀스의 최대 길이 8
프랑스어 시퀀스의 최대 길이 17


In [10]:
encoder_input = pad_sequences(encoder_input, maxlen = max_eng_seq_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen = max_fra_seq_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen = max_fra_seq_len, padding='post')

print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (33000, 8)
프랑스어 입력데이터의 크기(shape) : (33000, 17)
프랑스어 출력데이터의 크기(shape) : (33000, 17)


In [11]:
eng_to_index = eng_tokenizer.word_index
index_to_eng = eng_tokenizer.index_word

fra_to_index = fra_tokenizer.word_index
index_to_fra = fra_tokenizer.index_word

In [12]:
print(encoder_input[0])

[30  1  0  0  0  0  0  0]


In [13]:
print(decoder_input[0])

[ 1 91 12  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


In [14]:
print(decoder_target[0])

[91 12  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


In [ ]:
'''
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))
'''

In [15]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print(indices)

encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

[23575  7933  4082 ... 14577  1853  5291]


In [16]:
n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print('영어 학습데이터의 크기(shape) :',np.shape(encoder_input_train))
print('프랑스어 학습 입력데이터의 크기(shape) :',np.shape(decoder_input_train))
print('프랑스어 학습 출력데이터의 크기(shape) :',np.shape(decoder_target_train))

print('영어 테스트데이터의 크기(shape) :',np.shape(encoder_input_test))
print('프랑스어 테스트 입력데이터의 크기(shape) :',np.shape(decoder_input_test))
print('프랑스어 테스트 출력데이터의 크기(shape) :',np.shape(decoder_target_test))

영어 학습데이터의 크기(shape) : (30000, 8)
프랑스어 학습 입력데이터의 크기(shape) : (30000, 17)
프랑스어 학습 출력데이터의 크기(shape) : (30000, 17)
영어 테스트데이터의 크기(shape) : (3000, 8)
프랑스어 테스트 입력데이터의 크기(shape) : (3000, 17)
프랑스어 테스트 출력데이터의 크기(shape) : (3000, 17)


## 4. 임베딩 층 사용하기

In [17]:
latent_dim = 128

encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(eng_vocab_size, latent_dim)(encoder_inputs) # 임베딩 층
enc_masking = Masking(mask_value=0.0)(enc_emb) # 패딩 0은 연산에서 제외
encoder_lstm = LSTM(latent_dim, return_state=True) # 상태값 리턴을 위해 return_state는 True
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking) # 은닉 상태와 셀 상태를 리턴
encoder_states = [state_h, state_c] # 인코더의 은닉 상태와 셀 상태를 저장

## 5. 모델 구현하기

In [18]:
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(fra_vocab_size, latent_dim) # 임베딩 층
dec_emb = dec_emb_layer(decoder_inputs) # 패딩 0은 연산에서 제외
dec_masking = Masking(mask_value=0.0)(dec_emb)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True) 

decoder_outputs, _, _ = decoder_lstm(dec_masking, initial_state=encoder_states)

decoder_dense = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [19]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy", metrics=['acc'])
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 128)    596736      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 128)    937728      input_2[0][0]                    
_______________________________________________________________________________________

In [20]:
model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size=512, epochs=50)

Epoch 1/50
59/59 [==============================] - 7s 124ms/step - loss: 3.3711 - acc: 0.6023 - val_loss: 2.1646 - val_acc: 0.6131
Epoch 2/50
59/59 [==============================] - 6s 104ms/step - loss: 1.9858 - acc: 0.6480 - val_loss: 1.8417 - val_acc: 0.6773
Epoch 3/50
59/59 [==============================] - 6s 104ms/step - loss: 1.7521 - acc: 0.7260 - val_loss: 1.6758 - val_acc: 0.7356
Epoch 4/50
59/59 [==============================] - 6s 104ms/step - loss: 1.6251 - acc: 0.7389 - val_loss: 1.5910 - val_acc: 0.7438
Epoch 5/50
59/59 [==============================] - 6s 104ms/step - loss: 1.5500 - acc: 0.7440 - val_loss: 1.5295 - val_acc: 0.7461
Epoch 6/50
59/59 [==============================] - 6s 105ms/step - loss: 1.4874 - acc: 0.7488 - val_loss: 1.4746 - val_acc: 0.7534
Epoch 7/50
59/59 [==============================] - 6s 105ms/step - loss: 1.4233 - acc: 0.7580 - val_loss: 1.4183 - val_acc: 0.7631
Epoch 8/50
59/59 [==============================] - 6s 105ms/step - loss: 1.

In [21]:
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)
encoder_model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 128)         596736    
_________________________________________________________________
masking (Masking)            (None, None, 128)         0         
_________________________________________________________________
lstm (LSTM)                  [(None, 128), (None, 128) 131584    
Total params: 728,320
Trainable params: 728,320
Non-trainable params: 0
_________________________________________________________________


In [22]:
# 이전 time step의 hidden state를 저장하는 텐서
decoder_state_input_h = Input(shape=(latent_dim,))
# 이전 time step의 cell state를 저장하는 텐서
decoder_state_input_c = Input(shape=(latent_dim,))
# 이전 time step의 hidden state와 cell state를 하나의 변수에 저장
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2 = dec_emb_layer(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

decoder_outputs2 = decoder_dense(decoder_outputs2)

In [23]:
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, 
                      outputs=[decoder_outputs2] + decoder_states2)
decoder_model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 128)    937728      input_2[0][0]                    
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 128)]        0                                            
_______________________________________________________________________________________

In [24]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 정수 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = fra_to_index['<sos>']

    stop_condition = False
    decoded_sentence = ''

    # stop_condition이 True가 될 때까지 루프 반복
    # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정합니다.
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 단어로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_fra[sampled_token_index]

         # 현재 시점의 예측 단어를 예측 문장에 추가
        decoded_sentence += ' '+sampled_char

        # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
        if (sampled_char == '<eos>' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [25]:
def seq2eng(input_seq):
    temp = ''
    for i in input_seq :
        if(i!=0):
            temp = temp + index_to_eng[i] + ' '
    return temp

def seq2fra(input_seq):
    temp = ''
    for i in input_seq:
        if ((i!=0 and i!=fra_to_index['<sos>']) and i!=fra_to_index['<eos>']):
            temp = temp + index_to_fra[i] + ' '
    return temp

In [26]:
for seq_index in [3,50,100,300,1001]:
    input_seq = encoder_input_train[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    
    print(35 * "-")
    print("입력 문장: ", seq2eng(encoder_input_train[seq_index]))
    print("정답 문장: ", seq2fra(decoder_input_train[seq_index]))
    print("번역기가 번역한 문장: ", decoded_sentence[:-5])

-----------------------------------
입력 문장:  don t go in there . 
정답 문장:  ne rentre pas l dedans . 
번역기가 번역한 문장:   ne le d p te pas ! 
-----------------------------------
입력 문장:  i owe him my life . 
정답 문장:  je lui dois la vie . 
번역기가 번역한 문장:   je me dois une faveur . 
-----------------------------------
입력 문장:  we ve gone too far . 
정답 문장:  nous sommes all es trop loin . 
번역기가 번역한 문장:   nous sommes all es trop loin . 
-----------------------------------
입력 문장:  i remember her . 
정답 문장:  je me souviens d elle . 
번역기가 번역한 문장:   je me souviens de le dire . 
-----------------------------------
입력 문장:  come on grow up . 
정답 문장:  enfin arr tez d agir comme des enfants ! 
번역기가 번역한 문장:   reviens ici ! 
